In [197]:
import pandas as pd
import math

In [198]:
RIDERS = pd.read_csv("./docs/data/riders.csv")
RIDER_TEAMS = pd.read_csv("./docs/data/rider_teams.csv")
MANAGERS = pd.read_csv("./docs/data/managers.csv")
MANAGER_TEAMS = pd.read_csv("./docs/data/manager_teams.csv")
POINTS_SYSTEM = pd.read_csv("./docs/data/points_system.csv")
RACES = pd.read_csv("./docs/data/races.csv")
RESULTS_2023 = pd.read_csv("./docs/data/results/results_2023_full.csv")
RESULTS_2024 = pd.read_csv("./docs/data/results/results_2024_full.csv")
RESULTS_2025 = pd.read_csv("./docs/data/results/results_2025_full.csv")

In [199]:
# Riders owned by
RIDERS_OWNED_BY = MANAGER_TEAMS.groupby(["RiderName"])["ManagerName"].apply(", ".join).reset_index()

# Race categories
RACE_CATEGORIES = RACES[["RaceName_PCS", "RaceCategory"]]

# Relevant results
RESULTS_POINTS = RESULTS_2025[["RACE", "STAGE_ID", "STAGE_TYPE", "RANK", "RIDER"]]
RESULTS_POINTS = RESULTS_POINTS.rename(columns = {
    "RACE": "RaceName",
    "STAGE_ID": "StageID",
    "STAGE_TYPE": "StageType",
    "RANK": "Rank",
    "RIDER": "RiderName"
})
RESULTS_POINTS = RESULTS_POINTS.dropna(subset = ["StageID"])
RESULTS_POINTS = RESULTS_POINTS.loc[RESULTS_POINTS["StageType"] != "TTT"]

def convert_to_int(row):
    if row.isdigit():
        return int(row)
    else:
        return 0
    
def stage_or_gcresult(stage_id, race_category):
    if stage_id.startswith("stage"):
        return race_category.replace("Tour ", "Stage ")
    else:
        return race_category    

RESULTS_POINTS = pd.merge(RESULTS_POINTS, RACE_CATEGORIES, how = "inner", left_on = ["RaceName"], right_on = ["RaceName_PCS"])
RESULTS_POINTS["Rank"] = RESULTS_POINTS.apply(lambda row: convert_to_int(row["Rank"]), axis = 1)
RESULTS_POINTS["RaceCategory"] = RESULTS_POINTS.apply(lambda row: stage_or_gcresult(row["StageID"], row["RaceCategory"]), axis = 1)
RESULTS_POINTS = pd.merge(RESULTS_POINTS, POINTS_SYSTEM, how = "left", left_on = ["RaceCategory", "Rank"], right_on = ["RaceCategory", "RaceRank"])
RESULTS_POINTS = pd.merge(RESULTS_POINTS, RIDERS, how = "left", left_on = ["RiderName"], right_on = ["RiderName_PCS"])

# Rider points
RIDER_POINTS = RESULTS_POINTS.groupby(["RiderName_Zweeler"])["RacePoints"].sum().reset_index()

# Rider racedays
RIDER_RACEDAYS = RESULTS_POINTS.loc[RESULTS_POINTS["StageID"] != "gc"]
RIDER_RACEDAYS = RIDER_RACEDAYS.groupby(["RiderName_Zweeler"])["RiderName_Zweeler"].count().rename("Racedays").reset_index()
#RIDER_RACEDAYS.rename(columns = {0: "RiderName_Zweeler", 1: "Racedays"}, inplace = True)

In [200]:
RIDER_OUTPUT = RIDERS

RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDER_RACEDAYS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName_Zweeler"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDER_POINTS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName_Zweeler"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDERS_OWNED_BY, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName"])

def ppm(points, millions):
    if math.isnan(points):
        return round(0, 2)
    else:
        return round(points / millions, 1)

def float_to_int(row):
    if math.isnan(row):
        return 0
    else:
        return int(row)

RIDER_OUTPUT["PPM"] = RIDER_OUTPUT.apply(lambda row: ppm(row["RacePoints"], row["RiderPrice"]), axis = 1)
RIDER_OUTPUT["RacePoints"] = RIDER_OUTPUT.apply(lambda row: float_to_int(row["RacePoints"]), axis = 1)
RIDER_OUTPUT["Racedays"] = RIDER_OUTPUT.apply(lambda row: float_to_int(row["Racedays"]), axis = 1)

RIDER_OUTPUT = RIDER_OUTPUT[["RiderName_Zweeler", "RiderTeam", "RiderPrice", "Racedays", "RacePoints", "PPM", "ManagerName"]]

RIDER_OUTPUT.rename(columns = {
    "RiderName_Zweeler": "Navn",
    "RiderTeam": "Hold",
    "Racedays": "Løbsdage",
    "RacePoints": "Point",
    "PPM": "Profit",
    "ManagerName": "Ejet af"
})

# CREATE TABLE CONSISTING OF:
# RIDERNAME
# RIDERTEAM
# RACEDAYS TOTAL (Number from PCS)
# RACEDAYS IN GAME (NUmber of stages incl. DNF/DNS, excluding TTT and GC)
# Points 2025 (calculated)
# Points 2024 (calculated)
# PPM 2025 (calculated)
# Picked by number in game (need to scrape from Zweeler)
# Cheapo eligible (<= 2.5 + not banned)
# Managers owning

,Navn,Hold,RiderPrice,Løbsdage,Point,Profit,Ejet af
0,"Aberasturi, Jon",Euskaltel - Euskadi,2.7,0,0,0.0,NaN
1,"Abrahamsen, Jonas",Uno-X Mobility,6.2,5,0,0.0,Tommy
2,"Ackermann, Pascal",Israel - Premier Tech,6.6,0,0,0.0,NaN
3,"Adrià, Roger",Red Bull - BORA - hansgrohe,6.8,0,0,0.0,Okholm
4,"Aert, Wout van",Team Visma - Lease a Bike,37.4,0,0,0.0,"Chrelle, Jarma, Matti, Okholm, Tommy"
...,...,...,...,...,...,...,...
917,"Zingle, Axel",Team Visma - Lease a Bike,6.1,0,0,0.0,"Tommy, Visti"
918,"Zoccarato, Samuele",Team Polti VisitMalta,1.0,0,0,0.0,NaN
919,"Zubeldia, Unai",Euskaltel - Euskadi,1.0,0,0,0.0,NaN
920,"Zukowsky, Nick",Q36.5 Pro Cycling Team,1.0,5,0,0.0,NaN


In [201]:
"""
BANS
Kenk: Staune-Mittet
Visti: Jeanniere
Jarma: Rex
Jappo: Tim Tom Timtomberg
Chrelle: Withen
Tore: Goossens
Hustlersen: Kogut
Kenk: Aleotti
Visti: Seixas
Jarma: Uhlig
Jappo: Biermans
Chrelle: Brenner
Tore: Behrens
Hustlersen: Sam Watson
"""

'\nBANS\nKenk: Staune-Mittet\nVisti: Jeanniere\nJarma: Rex\nJappo: Tim Tom Timtomberg\nChrelle: Withen\nTore: Goossens\nHustlersen: Kogut\nKenk: Aleotti\nVisti: Seixas\nJarma: Uhlig\nJappo: Biermans\nChrelle: Brenner\nTore: Behrens\nHustlersen: Sam Watson\n'

In [202]:
html = RIDER_OUTPUT.to_html(table_id = "table1")

print(html)

<table border="1" class="dataframe" id="table1">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>RiderName_Zweeler</th>
      <th>RiderTeam</th>
      <th>RiderPrice</th>
      <th>Racedays</th>
      <th>RacePoints</th>
      <th>PPM</th>
      <th>ManagerName</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Aberasturi, Jon</td>
      <td>Euskaltel - Euskadi</td>
      <td>2.7</td>
      <td>0</td>
      <td>0</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Abrahamsen, Jonas</td>
      <td>Uno-X Mobility</td>
      <td>6.2</td>
      <td>5</td>
      <td>0</td>
      <td>0.0</td>
      <td>Tommy</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Ackermann, Pascal</td>
      <td>Israel - Premier Tech</td>
      <td>6.6</td>
      <td>0</td>
      <td>0</td>
      <td>0.0</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Adrià, Roger</td>
      <td>Red Bull - BORA - hansgrohe</td>